In [ ]:
# activate line execution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# plotly
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np

In [ ]:
# !pip install wandb -qqq
import os
os.environ["WANDB_API_KEY"] = "0f780ac8a470afe6cb7fc474ff3794772c660465"

os.environ["WANDB_NOTEBOOK_NAME"] = "jup_res"

import wandb
# wandb.login()
api = wandb.Api()





# selected_tag = 'Pred'

runs = api.runs("hokarami/TEEDAM_supervised")


In [37]:
import pandas as pd 


# Project is specified by <entity/project-name>
def dl_runs(all_runs, selected_tag=None):




    summary_list, config_list, name_list, path_list = [], [], [], []
    for run in all_runs: 

        if (selected_tag not in run.tags) and (selected_tag is not None):
            continue
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files 
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        

        config_list.append(
            {k: v for k,v in run.config.items()
            if not k.startswith('_')})

        # .name is the human-readable name of the run.
        name_list.append(run.name)
        path_list.append(run.path)

    runs_df = pd.DataFrame({
        "summary": summary_list,
        "config": config_list,
        "name": name_list,
        "path": path_list

        })

    # runs_df.to_csv("project.csv")
    return runs_df

In [ ]:
# q = runs_df['name'].str.contains('\[R2-')
# runs_df = runs_df[q]


runs_df = dl_runs(runs, selected_tag='H20')

runs_df
runs_df.iloc[0].summary.keys()
runs_df.iloc[0].config.keys()

# Table I
Effect of time concatenation and single+mark

In [ ]:
runs = api.runs("hokarami/TEEDAM_unsupervised_timeCat")
df_filt = dl_runs(runs, selected_tag='H70')
df_filt


In [ ]:
i_groupby_row =['dataset']
i_groupby_col = ['time_enc']
i_groupby_col=['mod','diag_offset','time_enc']
i_agg = ['split']
i_metric = ['Best-Test-CIF/LL-#events', 'Best-Test-NextType(ML)/auc-ovo-weighted','Best-Test-NextType(MC)/f1-weighted']
dict_rename_metrics = {'Best-Test-CIF/LL-#events':'LL/#events', 'Best-Test-NextType(ML)/auc-ovo-weighted':'AUROC','Best-Test-NextType(MC)/f1-weighted':'F1-Score'}
# i_metric = ['Test-CIF/NLL-#events']
i_stats=['mean','std']
# i_stats=['mean']

#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)

# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]

df = df.rename(columns=dict_rename_metrics)
i_metric = list(dict_rename_metrics.values())

# df = df.melt(id_vars=i_groupby_row + i_groupby_col + i_agg ,value_vars=i_metric, var_name='Metric')




a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(i_stats).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:str(round(x.mean(),2))  + ' (' + str(round(x.std(),2))  +')'  ).reset_index() #  

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()

a1.loc[   a1['mod'].isin(['ml','mc']) , 'mod']='PP(ML/MC)'
a1.loc[   a1['mod'].isin(['none']) , 'mod']='AE'
a1.loc[   a1['mod'].isin(['single']) , 'mod']='PP(single)'

a1



df_result = a1.melt(id_vars=i_groupby_row + i_groupby_col,value_vars=i_metric, var_name='Metric')



# df = df.melt(id_vars=i_groupby_row + i_groupby_col + i_agg ,value_vars=i_metric, var_name='Metric')

df_result = df_result.pivot(index=i_groupby_row+['Metric'],columns=i_groupby_col)

# remove irrelevant metrics
df_result = df_result.replace('nan (nan)', np.nan).dropna(axis=0,how='all')
df_result


In [ ]:
df_result.transpose().sort_values(by=i_groupby_col,axis=0)

# Table II

In [38]:
runs = api.runs("hokarami/TEEDAM_unsupervised")
df_filt = dl_runs(runs, selected_tag='H70')
df_filt

,summary,config,name,path
0,"{'Test-CIF/#events': 16165, 'Test-CIF/LL-#even...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TEnoise__pp_single_mark-concat-d1]1813060,"[hokarami, TEEDAM_unsupervised, o37u78ad]"
1,"{'Valid-NextTime/RMSE': 3.261590011398012, 'Te...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TE__pp_single_mark-concat-d1]1818711,"[hokarami, TEEDAM_unsupervised, eivjntvr]"
2,"{'_wandb': {'runtime': 352}, 'Test-CIF/LL-#eve...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TEnoise__pp_single_mark-concat-d1]1812621,"[hokarami, TEEDAM_unsupervised, x9ll87wv]"
3,"{'Valid-NextTime/msae': 1.4494839811915183, 'T...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TEDA__pp_single_mark-concat-d1]1817732,"[hokarami, TEEDAM_unsupervised, mmdwiu0a]"
4,"{'max_obj_val': 0.11251283740550688, 'Valid-CI...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TE__pp_single_mark-concat-d1]1812797,"[hokarami, TEEDAM_unsupervised, boutyub6]"
...,...,...,...,...
535,"{'Valid-CIF/#events': 203065, 'Valid-NextTime/...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TE__pp_single_mark-concat-d-2]1804673,"[hokarami, TEEDAM_unsupervised, ekltqref]"
536,"{'Valid-NextTime/msae': 1.2931187526090924, '_...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TEDA__pp_single_mark-concat-d-2]1808225,"[hokarami, TEEDAM_unsupervised, ahjhqxzc]"
537,{'Valid-NextType(ML)/f1-weighted': 0.273110995...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TE__pp_single_mark-concat-d-2]1803647,"[hokarami, TEEDAM_unsupervised, mrex3brg]"
538,{'Best-Valid-NextType(ML)/auc-ovo-weighted': 0...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TEnoise__pp_single_mark-concat-d-2]1802207,"[hokarami, TEEDAM_unsupervised, fsd0h7d7]"


In [ ]:
i_groupby_row =['dataset']
i_groupby_col = ['time_enc']
i_groupby_col=['setting','INPUT','diag_offset']
i_agg = ['split']
i_metric = ['Best-Test-CIF/LL-#events', 'Best-Test-NextType(ML)/auc-ovo-weighted']
dict_rename_metrics = {'Best-Test-CIF/LL-#events':'LL/#events', 'Best-Test-NextType(ML)/auc-ovo-weighted':'AUROC'}
# i_metric = ['Test-CIF/NLL-#events']
i_stats=['mean','std']
# i_stats=['mean']

#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)

# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]

df = df.rename(columns=dict_rename_metrics)
i_metric = list(dict_rename_metrics.values())

# df = df.melt(id_vars=i_groupby_row + i_groupby_col + i_agg ,value_vars=i_metric, var_name='Metric')




a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(i_stats).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:str(round(x.mean(),2))  + ' (' + str(round(x.std(),2))  +')'  ).reset_index() #  


# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()

# a1.loc[   a1['mod'].isin(['ml','mc']) , 'mod']='PP(ML/MC)'
# a1.loc[   a1['mod'].isin(['none']) , 'mod']='AE'
# a1.loc[   a1['mod'].isin(['single']) , 'mod']='PP(single)'

a1



df_result = a1.melt(id_vars=i_groupby_row + i_groupby_col,value_vars=i_metric, var_name='Metric')



# df = df.melt(id_vars=i_groupby_row + i_groupby_col + i_agg ,value_vars=i_metric, var_name='Metric')

df_result = df_result.pivot(index=i_groupby_row+['Metric'],columns=i_groupby_col)

# remove irrelevant metrics
df_result = df_result.replace('nan (nan)', np.nan).dropna(axis=0,how='all')
df_result


In [36]:
df_filt

,summary,config,name
0,"{'Test-CIF/#events': 16165, 'Test-CIF/LL-#even...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TEnoise__pp_single_mark-concat-d1]1813060
1,"{'Valid-NextTime/RMSE': 3.261590011398012, 'Te...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TE__pp_single_mark-concat-d1]1818711
2,"{'_wandb': {'runtime': 352}, 'Test-CIF/LL-#eve...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TEnoise__pp_single_mark-concat-d1]1812621
3,"{'Valid-NextTime/msae': 1.4494839811915183, 'T...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TEDA__pp_single_mark-concat-d1]1817732
4,"{'max_obj_val': 0.11251283740550688, 'Valid-CI...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TE__pp_single_mark-concat-d1]1812797
...,...,...,...
535,"{'Valid-CIF/#events': 203065, 'Valid-NextTime/...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TE__pp_single_mark-concat-d-2]1804673
536,"{'Valid-NextTime/msae': 1.2931187526090924, '_...","{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TEDA__pp_single_mark-concat-d-2]1808225
537,{'Valid-NextType(ML)/f1-weighted': 0.273110995...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TE__pp_single_mark-concat-d-2]1803647
538,{'Best-Valid-NextType(ML)/auc-ovo-weighted': 0...,"{'w': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[H70-TEnoise__pp_single_mark-concat-d-2]1802207


In [48]:
runs = api.runs("hokarami/TEEDAM_unsupervised")
df_filt = dl_runs(runs, selected_tag='H70')
df_filt

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )
df_path = df_filt.path.apply(lambda x:'/'.join(x))

df = pd.concat([df_config, df_summary, df_path],axis=1)

q=(df['dataset']=='P12') & (df['setting']=='mc2') & (df['INPUT']=='TEDAM') & (df['diag_offset']==1) & (df['test_center']=='1')
# df[q]
# list(df.columns)
df.loc[q,'path']

115    hokarami/TEEDAM_unsupervised/zo2u72u2
Name: path, dtype: object

# Table III

In [ ]:
df_filt = dl_runs(runs, selected_tag='H50')


In [ ]:
i_groupby_row =['dataset','setting','test_center']
i_groupby_col = ['event_enc','mod']
i_groupby_col=['INPUT','OUTPUT']
i_agg = ['split']
i_metric = ['Best-Test-pred_label/f1-binary','Best-Test-pred_label/AUPRC','Best-Test-pred_label/AUROC']
# i_metric = ['Test-CIF/NLL-#events']
i_stats=['mean','std']
# i_stats=['mean']

#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)
# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]
df




a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(i_stats).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:str(round(x.mean(),2))  + ' (' + str(round(x.std(),2))  +')'  ) #  

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1

a1.reset_index().pivot(index=i_groupby_row,columns=i_groupby_col)



# a1.to_excel("aaa.xlsx")

In [ ]:
df

# CIF

In [ ]:
df_filt = dl_runs(runs, selected_tag='CIF')

df_filt.iloc[0].config.keys()

In [ ]:
i_groupby_row =['dataset','setting',]
i_groupby_col = ['state']
i_agg = ['test_center','split']
# i_metric = ['Best-Test-pred_label/f1-binary','Best-Test-pred_label/AUPRC']
i_metric = ['Test-CIF/NLL-#events']


#df_filt.iloc[0].config.keys()

d_name = [x for x in df_filt.name]

df_config = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.config]    )

df_summary = pd.DataFrame(   [{k:v for k,v in x.items()} for x in df_filt.summary]    )

df = pd.concat([df_config, df_summary],axis=1)#.dropna(axis=1,how='all')
# df.name = df_filt.name
df = df[i_groupby_row + i_groupby_col + i_agg + i_metric]
df





a0 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].size().rename('runs')

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(['mean','std']).round(3)

a1 = df.groupby(i_groupby_row + i_groupby_col)[i_metric].aggregate(lambda x:str(round(x.mean(),3))  + ' (' + str(round(x.std(),3))  +')' )

# a4 = df.groupby(i_by)[i_metric].apply(lambda x:x.round(3).tolist())   # x.round(3).tolist()
a1

a1.reset_index().pivot(index=i_groupby_row,columns=i_groupby_col)



# a1.to_csv("aaa.csv")